In [1]:
from pyqcu import io, gauge, cg, bistabcg, define, qcu, linalg
from pyqcu.set import *
import cupy as cp
import numpy as np
import time
import matplotlib.pyplot as plt
import cupy as cp
import numpy as np
np.Inf = np.inf


    @@@@@@######QCU NOTES START######@@@@@@@
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (wilson:x=4,y=4,z=4,t=4;clover:x=8,y=8,z=8,t=8) that QCU support (when '#define _BLOCK_SIZE_ 

In [2]:
params[define._LAT_X_] = 32
params[define._LAT_Y_] = 16
params[define._LAT_Z_] = 4
params[define._LAT_T_] = 4
params[define._LAT_XYZT_] = params[define._LAT_X_] * \
    params[define._LAT_Y_] * params[define._LAT_Z_] * params[define._LAT_T_]
params[define._LAT_E_] = 4
params[define._NODE_RANK_] = define.rank
params[define._NODE_SIZE_] = define.size
params[define._DATA_TYPE_] = define._LAT_C128_
argv[define._MASS_] = 0.0
argv = argv.astype(define.dtype(params[define._DATA_TYPE_])).real
kappa = 1 / (2 * argv[define._MASS_] + 8)
print('My rank is ', define.rank)
print("Parameters:", params)
print("Args:", argv)
wilson_cg_params = params.copy()
wilson_cg_params[define._SET_INDEX_] = 0
wilson_cg_params[define._SET_PLAN_] = define._SET_PLAN1_
qcu.applyInitQcu(set_ptrs, wilson_cg_params, argv)
wilson_dslash_eo_params = params.copy()
wilson_dslash_eo_params[define._SET_INDEX_] = 1
wilson_dslash_eo_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_eo_params[define._PARITY_] = define._EVEN_
wilson_dslash_eo_params[define._DAGGER_] = define._NO_USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_eo_params, argv)
wilson_dslash_eo_dag_params = params.copy()
wilson_dslash_eo_dag_params[define._SET_INDEX_] = 2
wilson_dslash_eo_dag_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_eo_dag_params[define._PARITY_] = define._EVEN_
wilson_dslash_eo_dag_params[define._DAGGER_] = define._USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_eo_dag_params, argv)
wilson_dslash_oe_params = params.copy()
wilson_dslash_oe_params[define._SET_INDEX_] = 3
wilson_dslash_oe_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_oe_params[define._PARITY_] = define._ODD_
wilson_dslash_oe_params[define._DAGGER_] = define._NO_USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_oe_params, argv)
wilson_dslash_oe_dag_params = params.copy()
wilson_dslash_oe_dag_params[define._SET_INDEX_] = 4
wilson_dslash_oe_dag_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_oe_dag_params[define._PARITY_] = define._ODD_
wilson_dslash_oe_dag_params[define._DAGGER_] = define._USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_oe_dag_params, argv)
print("Set pointers:", set_ptrs)
print("Set pointers data:", set_ptrs.data)

My rank is  0
Parameters: [   32    16     4     4  8192     1     1     1     1     0     0     1
     0 10000     4     0     0     4     4     4     8     4]
Args: [0.00000000e+00 9.99999972e-10]
gridDim.x               :128
blockDim.x              :32
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :16
host_params[_LAT_Z_]    :4
host_params[_LAT_T_]    :4
host_params[_LAT_XYZT_] :4096
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_DATA_TYPE_]:4
host_params[_SET_INDEX_]:0
host_params[_SET_PLAN_] :1
host_params[_MG_X_]     :4
host_params[_MG_Y_]     :4
host_params[_MG_Z_]     :4
host_params[_MG_T_]     :8
host_params[_LAT_E_]    :4
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :0.000000e+00
lat_2dim[_XY_]          :256
lat_2dim[_XZ_]          :64
lat_2dim

In [3]:
argv

array([0.00000000e+00, 9.99999972e-10])

In [4]:
U = gauge.give_gauss_SU3(sigma=0.1, seed=12138,
                         dtype=define.dtype(params[define._DATA_TYPE_]), size=params[define._LAT_XYZT_]*define._LAT_S_)
U = io.dptzyxcc2ccdptzyx(
    io.gauge2dptzyxcc(gauge=U, params=params))

U_size = 32768
U_0 is ready.

U_1 is ready.

U_2 is ready.

U_3 is ready.

U_4 is ready.

U_5 is ready.

U_6 is ready.

U_7 is ready.

U_8 is ready.

U_9 is ready.

U_10 is ready.

U_11 is ready.

U_12 is ready.

U_13 is ready.

U_14 is ready.

U_15 is ready.

U_16 is ready.

U_17 is ready.

U_18 is ready.

U_19 is ready.

U_20 is ready.

U_21 is ready.

U_22 is ready.

U_23 is ready.

U_24 is ready.

U_25 is ready.

U_26 is ready.

U_27 is ready.

U_28 is ready.

U_29 is ready.

U_30 is ready.

U_31 is ready.

U_32 is ready.

U_33 is ready.

U_34 is ready.

U_35 is ready.

U_36 is ready.

U_37 is ready.

U_38 is ready.

U_39 is ready.

U_40 is ready.

U_41 is ready.

U_42 is ready.

U_43 is ready.

U_44 is ready.

U_45 is ready.

U_46 is ready.

U_47 is ready.

U_48 is ready.

U_49 is ready.

U_50 is ready.

U_51 is ready.

U_52 is ready.

U_53 is ready.

U_54 is ready.

U_55 is ready.

U_56 is ready.

U_57 is ready.

U_58 is ready.

U_59 is ready.

U_60 is ready.

U_61 is ready.

U_6

In [5]:
def pdslash_no_dag(src):
    tmp0 = cp.zeros_like(src)
    tmp1 = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        tmp0, src, U, set_ptrs, wilson_dslash_eo_params)
    qcu.applyWilsonDslashQcu(
        tmp1, tmp0, U, set_ptrs, wilson_dslash_oe_params)
    return src-kappa**2*tmp1


def pdslash_dag(src):
    tmp0 = cp.zeros_like(src)
    tmp1 = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        tmp0, src, U, set_ptrs, wilson_dslash_eo_dag_params)
    qcu.applyWilsonDslashQcu(
        tmp1, tmp0, U, set_ptrs, wilson_dslash_oe_dag_params)
    return src-kappa**2*tmp1


def dslash_no_dag(src):
    dest = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        dest, src, U, set_ptrs, wilson_dslash_eo_params)
    return dest


def dslash_dag(src):
    dest = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        dest, src, U, set_ptrs, wilson_dslash_eo_dag_params)
    return dest


def bistabcg_dslash(src):
    return pdslash_no_dag(src)


def cg_dslash(src):
    return pdslash_dag(pdslash_no_dag(src))

In [6]:
src = cp.zeros(
    params[define._LAT_XYZT_]*define._LAT_SC_, dtype=define.dtype(params[define._DATA_TYPE_]))
src = linalg.initialize_random_vector(src)
src=io.fermion2ptzyxsc(src, params=params)
dest = cp.zeros_like(src)
qcu.applyWilsonBistabCgQcu(dest, src,
                           U, set_ptrs, wilson_cg_params)
# qcu.applyWilsonCgQcu(dest, src,
#                      U, set_ptrs, wilson_cg_params)

##RANK:0##LOOP:9999##Residual:(-nan,-nani)
multi-gpu wilson bistabcg total time: (without malloc free memcpy) :12.323312746 sec
######TIME  :85386######
##RANK      :0
##LOOP      :999
##tmp0      :(-nan,-nani)
##tmp1      :(-nan,-nani)
##rho_prev  :(-nan,-nani)
##rho       :(-nan,-nani)
##alpha     :(-nan,-nani)
##beta      :(-nan,-nani)
##omega     :(-nan,-nani)
##send_tmp  :(-nan,-nani)
##norm2_tmp :(0.614784,4.00632e-20i)
##diff_tmp  :(-nan,-nani)
##lat_4dim  :(4096,0i)
